In [0]:
from pyspark.sql.functions import *
import re

In [0]:
#Routes
input_path = "/Volumes/workspace/default/data_stage/"
checkpoint = "/Volumes/workspace/default/checkpoints/bronze/data_prod_cp"
schema_location = "/Volumes/workspace/default/schema/bronze/data_prod_schema"

In [0]:
%sql
-- Create volumes for landing, checkpointing and schema tracking
CREATE VOLUME IF NOT EXISTS workspace.default.data_stage COMMENT 'landing_zone';
CREATE VOLUME IF NOT EXISTS workspace.default.checkpoints COMMENT 'Checkpoint storage';
CREATE VOLUME IF NOT EXISTS workspace.default.schema_tracking COMMENT 'Schema tracking for Auto Loader';

-- Create delta table for bronze table 
CREATE TABLE IF NOT EXISTS workspace.default.bronze_pmx_prod
USING DELTA;

#### Landing to Bronze

In [0]:
#Read streaming table with cloudFiles Format
df_raw_stream = (
    spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format", "csv")
    .option("cloudFiles.inferColumnTypes", "true") #Infer column type for new columns 
    .option("cloudFiles.schemaLocation","/Volumes/workspace/default/schema_tracking")
    .option("header", "true")
    .option("cloudFiles.schemaEvolutionMode", "addNewColumns") #Add new columns to schema
    .load(input_path)
)


#Normalize columns
def normalize_cols(df_raw_stream,cols):
    df_rename_col = df_raw_stream\
        .select(
            [col(c).alias(re.sub(r"\s+", "_", re.sub(r"\s*\(.*\)", "", c).strip())) for c in raw_cols] # Rename columns using regex
        )
    return df_rename_col

# Adding new columns for date and ingestion date
def add_ingestion_date(df_rename_col):
    return df_rename_col.withColumn("ingestion_date", current_timestamp())


raw_cols = df_raw_stream.columns #Get columns from raw table
df_rename = normalize_cols(df_raw_stream,raw_cols)
df_transformed = add_ingestion_date(df_rename)

# write stream
query = (
    df_transformed.writeStream
    .format("delta")
    .option("checkpointLocation", "/Volumes/workspace/default/checkpoints") 
    .option("mergeSchema", "true")
    .trigger(once=True) 
    .outputMode("append")
    .table("default.bronze_pmx_prod")
)

#### Bronze to Silver

In [0]:
df_bronze = spark.readStream.table("default.bronze_pmx_prod")

# Create functions for normaliza table and unpivot
def normalize_fecha(df):
    return (df
        .withColumn("_ym", F.to_date(F.concat_ws("-", F.col("FECHA"), F.lit("01")), "yyyy-MM-dd"))
        .withColumn("FECHA", F.last_day(F.col("_ym")))
        .drop("_ym")
    )
def unpivot_to_long(df_normalize):
    cols = [c for c in df_normalize.columns if c != "FECHA"]
    N = len(cols)
    pairs = ", ".join([f"'{c}', `{c}`" for c in cols])
    unpivot_df = df_normalize\
        .select(
            col("FECHA"),
            expr(f"stack({N}, {pairs}) AS (POZO, PRODUCION_MBD)")
        )

    return unpivot_df

def add_flags(df_unpivot):
    withColumn("PRODUCION_MBD", col("PRODUCION_MBD").cast("double"))\
    .withColumn("FLAG_NULL", when(col("PRODUCION_MBD")).IsNull(),1).otherwise(0)\
    .withColumn("FLAG_ZERO", when(col("PRODUCION_MBD") == 0),1).otherwise(0)
    return df_unpivot



In [0]:
%sql
CREATE TABLE IF NOT EXISTS default.silver_pmx_prod (
  FECHA DATE,
  ANIO INT,
  MES INT,
  POZO STRING,
  PRODUCCION_MBD DOUBLE,
  FLAG_NULL INT,
  FLAG_ZERO INT,
  FLAG_OUTLIER_IQR INT,
  FLAG_OUTLIER_MOM INT,
  FLAG_INACTIVO INT,
  DATA_SOURCE STRING,
  INGESTION_TS TIMESTAMP
) USING DELTA PARTITIONED BY (ANIO, MES)

In [0]:
def process_silver(microDF, batchId):
    if microDF.rdd.isEmpty():
        return

    # Normalize and unpivot table    
    df_normalize = normalize_fecha(microDF)
    df_unpivot = unpivot_to_long(df_normalize)
    df = add_flags(df_unpivot)

    #Collect min and max date
    minmax = df.agg(min("FECHA").alias("minf"),max("FECHA").alias("maxf")).collect()[0]
    minf, maxf = minmax["minf"], minmax["maxf"]

    hist = spark.table(silver_table).filter(
        (col("FECHA") >=add_months(lit(minf), -12)) & (col("FECHA") <= add_months(lit(maxf), 0))
    ).select("POZO", "FECHA", "PRODUCION_MBD")

    union_for_stats = df.select("POZO", "PRODUCCION_MBD").unionByName(
        hist.selec("POZO","PRODUCCION_MBD")
    )

    stats = (
        union_for_stats
        .filter(col("PRODUCCION_MBD").isNotNull())
        .groupBy("POZO")
        .agg(
            expr("percentile_approx(PRODUCCION_MBD, 0.25, 1000)").alias("Q1"),
            expr("percentile_approx(PRODUCCION_MBD, 0.75, 1000)").alias("Q3")
        )
        .withColumn("IQR", F.col("Q3") - F.col("Q1"))
        .withColumn("LOW_B", F.col("Q1") - 1.5*F.col("IQR"))
        .withColumn("HIGH_B", F.col("Q3") + 1.5*F.col("IQR"))
    )
    
